In [3]:
import cv2
import time
import numpy as np
from detector import YOLOv8
from utils import compute_iou, compute_intersection, compute_polygon_intersection
import matplotlib.pyplot as plt
import os
import time

In [12]:
start_time = time.time()
filename = './temp.txt'

duration = 5
time.sleep(4)

if (time.time() - start_time < duration):
    if os.path.isfile(filename):
        os.remove(filename)

In [75]:
from shapely.geometry import Polygon

def test(box, boxes):
    # Создайте объекты Polygon для четырёхугольника и прямоугольника
    x1 = box[0]
    y1 = box[1]
    x2 = box[2]
    y2 = box[3]

    poly1 = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) # Четырёхугольник
    
    intersections = []
    for temp_box in boxes:
        poly2 = Polygon(temp_box) # полигон

        # Найдите пересечение двух полигонов
        intersection = poly1.intersection(poly2)

        # Найдите площадь пересечения
        intersections.append(intersection.area / poly1.area)

    return intersections


print(test([50, 50, 150, 150], parking_areas))

[0.6189952153110048, 0.0]


In [31]:
zones = [('223,1 333,148 156,334 34,322 14,149',), ('512,30 617,13 698,86 622,183',)]
parking_areas = []
for zone in zones:
    bbox = [list(map(int, elem.split(','))) for elem in zone[0].split(' ')]
    # print(bbox)
    parking_areas.append(bbox)

# parking_areas = np.array(parking_areas).astype('int')
parking_areas

[[[223, 1], [333, 148], [156, 334], [34, 322], [14, 149]],
 [[512, 30], [617, 13], [698, 86], [622, 183]]]

In [76]:
a = ['223', '1']
b = ['136', '334']
a.append(b)
a

TypeError: list.append() takes no keyword arguments

In [ ]:
gen(r'http://192.168.0.39:8080/video')

In [8]:
def gen(address):
    """
    Запуск модели
    """
    # Засекаем начало работы скрипта
    start_time = time.time()
    # Длительность нахождения
    duration = 30

    # Захват видео с камеры
    # cap = cv2.VideoCapture('./image.jpg')
    
    model_path = r'./yolov8n.onnx'
    yolov8_detector = YOLOv8(path=model_path,
                             conf_thres=0.3,
                             iou_thres=0.5)

    cv2.namedWindow('stream', cv2.WINDOW_NORMAL)
    cap = cv2.VideoCapture(address)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (720,480))
    
    # Координаты территории
    txt = './parking_areas.txt'
    lines = None
    with open(txt) as f:
        lines = f.readlines()
    bboxes = []
    for line in lines:
        line = line.replace('\n', '').split(', ')
        bbox = [int(l) for l in line]
        bboxes.append(bbox)
    parking_areas = np.array(bboxes)

    # Сколько кадров подряд с пустым местом мы уже видели
    free_frames = 0
    
    while (time.time() - start_time < duration) and cap.isOpened():
        # Кадр с камеры
        ret, frame = cap.read()
        if not ret:
            break

        # Детектирование
        yolov8_detector(frame)
        # detected_img = frame
        detected_img = yolov8_detector.draw_detections(frame)
        bounding_boxes = np.array(yolov8_detector.get_boxes())
        if detected_img is None:
            continue
        
        # чтобы не ломалось iou
        if len(bounding_boxes) == 1 or bounding_boxes.shape[0] == 1:
            # bounding_boxes = np.array([bounding_boxes])
            bounding_boxes = np.array(bounding_boxes).reshape(1, -1)

        free_frames_temp = 0
        if bounding_boxes.shape[0] != 0:
            for bounding_box in bounding_boxes:
                intersections = compute_intersection(bounding_box, parking_areas)
                print(intersections)
                max_intersection = np.max(intersections)
                # Для отладки параметра IoU
                # print(parked_drums_boxes[i])
                # print(max_IoU)
                if max_intersection > 0.35:
                    free_frames = 0
                    continue
                else:
                    free_frames_temp += 1
        else:
            free_frames += 1
            
        if free_frames_temp == bounding_boxes.shape[0]:
                free_frames += 1

        print(f'{time.time() - start_time}')

        if free_frames > 100:
            break

        cv2.imshow('stream', detected_img)
        out.write(detected_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cv2.destroyAllWindows()
    out.release()
    cap.release()

In [10]:
start_time = time.time()
time.sleep(1)
finish_time = time.time()
print(f"Время выполнения: {finish_time - start_time:.5f}")

Время выполнения: 1.00119


In [6]:
cv2.namedWindow('stream', cv2.WINDOW_NORMAL)

cap = cv2.VideoCapture(r'http://192.168.0.39:8080/video')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (720,480))

while (cap.isOpened()):
    ret, frame = cap.read()

    if not ret:
        break

    out.write(frame)

    cv2.imshow('stream', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        out.release()
        cap.release()
        break

In [15]:
cap = cv2.VideoCapture(r'http://192.168.0.39:8080/video')
cv2.namedWindow('stream', cv2.WINDOW_NORMAL)

while cap.isOpened():
    ret, frame = cap.read()

    cv2.imshow('stream', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [3]:
model_path = r'./yolov8n.onnx'

yolov8_detector = YOLOv8(path=model_path,
                            conf_thres=0.3,
                            iou_thres=0.5)

frame = cv2.imread('./photo.jpg')

# cv2.imshow('frame', frame)
# while (True):
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         cv2.destroyAllWindows()
#         break

yolov8_detector(frame)
detected_img = yolov8_detector.draw_detections(frame)

cv2.imshow('detected_img', detected_img)
while (True):
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [35]:
cv2.imshow('frame', frame)
while (True):
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

## ------------

In [ ]:
txt = './parking_areas.txt'
lines = None
with open(txt) as f:
    lines = f.readlines()
bboxes = []
for line in lines:
    line = line.replace('\n', '').split(', ')
    bbox = [int(l) for l in line]
    bboxes.append(bbox)
parking_areas = np.array(bboxes)

In [ ]:
parking_areas = []
for zone in zones:
    bbox = zone[0].split(' ')[0].split(',')
    [bbox.append(elem) for elem in zone[0].split(' ')[2].split(',')]
    parking_areas.append(bbox)
    # parking_areas = np.append(parking_areas, bbox, axis=0)
parking_areas = np.array(parking_areas).astype('int')
print(parking_areas)